In [1]:
from IPython.display import display, Math, Latex, HTML
HTML('''<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }
  
  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>''') 

## 2.Summary

In [2]:
#Load libraries:
from data_exploration import *

In [3]:
all_data=False

In [4]:
#Collect all data?
#all_data=True

#Set up test time interval:
time_interval='4w' #2w

#Set up starting point, by default if will start from current time
starting_point=datetime.now().strftime('%Y-%m-%d %H:%M:%S')
#starting point="2019-01-10 14:00:00"  # to set upl alternative starting point

In [5]:
if not all_data:#Set up test time interval:
    print("Time interval: ", time_interval)
    print("Starting point:",starting_point )
else:
    print("Selecting all data from database")

Time interval:  4w
Starting point: 2019-01-24 13:59:09


In [6]:
#Set up influxdb connection:
client, client_df = connect_to_influxdb()

### Download speed

In [7]:
if not all_data:
    query_download = "SELECT * FROM SPEEDTEST_DOWNLOAD WHERE PROVIDER!='iperf' AND time >= '"+starting_point+"'-"+time_interval+" AND DOWNLOAD>0;"
else:
    query_download = "SELECT * FROM SPEEDTEST_DOWNLOAD WHERE PROVIDER!='iperf' AND DOWNLOAD>0;"
download_df = get_dataframe_from_influxdb(client_df=client_df,query_influx=query_download,table_name='SPEEDTEST_DOWNLOAD')
#download_df.head()
download_df=download_df[download_df["SK_PI"]!=3]
download_df['time']= download_df['time'].dt.tz_localize('UTC').dt.tz_convert('America/Winnipeg')

In [8]:
device_numbers_d=download_df['SK_PI'].unique()
device_numbers_d=list(map(int, device_numbers_d))
device_numbers_d= sorted(device_numbers_d)
#device_numbers

In [9]:
download_df["hour"]=pd.to_numeric(download_df["time"].dt.hour)

In [10]:
by_hour_by_device_d = mean_max_median_min_by2(input_dataframe=download_df,value1="DOWNLOAD", value2="DOWNLOAD",
                                              value3="DOWNLOAD",value4="DOWNLOAD",group_by_value="hour", rename_columns=True)
download_summary_by_hour=mean_max_median_min_by1(download_df,'DOWNLOAD',index_col='hour')
download_summary=mean_max_median_min_by1(download_df,'DOWNLOAD')

In [11]:
by_hour_by_device_d = mean_max_median_min_by2(input_dataframe=download_df,value1="DOWNLOAD", value2="DOWNLOAD",
                                              value3="DOWNLOAD",value4="DOWNLOAD",group_by_value="hour", rename_columns=True)

In [12]:
traces=[]
for device in device_numbers_d:
    #subset=by_hour_by_device_d[by_hour_by_device_d["SK_PI"]==device]
    subset=download_df[download_df["SK_PI"]==device]
    trace = go.Scatter(
        x = subset['hour'],
        #y=subset['DOWNLOAD'],
        y=(subset['DOWNLOAD']-subset['DOWNLOAD'].mean())/subset['DOWNLOAD'].std(),
        #y=subset['mean'],
        #y=(subset['mean']-subset['mean'].mean())/subset['mean'].std(),
        mode = 'markers',
        marker = dict(color=colors[device]),
        name = device
    )
    traces.append(trace)
layout = go.Layout(
        title="Average download speed by hour over the last "+time_interval+ " starting from "+ starting_point,
        xaxis=dict(title="hour"),
        yaxis=dict(title=" ")
        )
#download_line=go.Scatter(x=by_hour_by_device_d['hour'],y=[50] * len(by_hour_by_device_d['hour']), mode='lines',marker=dict(color='red'), name='50Mbps')
#traces.append(download_line)
data = traces
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [13]:
if all_data:
    t="Download speed"
else:
    t="Download speed by hour over the last "+time_interval+ " starting from "+ starting_point,
simple_boxplot(dataframe=download_df,plot_value='DOWNLOAD',sort_value='SK_PI',
               title=t, 
               ytitle="Mbps",downloadline=True)

In [14]:
download_df["below50"]=0
download_df.loc[(download_df["DOWNLOAD"]<50),"below50"]=1
#download_df.head()

In [15]:
summary_download=download_df.groupby("SK_PI")['below50'].count().reset_index()
summary_download.rename(columns={'below50':'download_count'}, inplace=True)

mean_download=round(download_df.groupby("SK_PI")['DOWNLOAD'].mean(),2).reset_index()
mean_download.rename(columns={'DOWNLOAD':'download_mean'}, inplace=True)

std_download=round(download_df.groupby("SK_PI")['DOWNLOAD'].std(),2).reset_index()
std_download.rename(columns={'DOWNLOAD':'download_std'}, inplace=True)

subset_below50=download_df[download_df['below50']==1]
summary_below50=subset_below50.groupby("SK_PI")['below50'].count().reset_index()
summary_below50.rename(columns={'below50':'download_below50'}, inplace=True)

summary_download=pd.merge(summary_download, summary_below50,  how='outer', left_on=['SK_PI'], right_on = ['SK_PI'])
summary_download=pd.merge(summary_download, mean_download,  how='outer', left_on=['SK_PI'], right_on = ['SK_PI'])
summary_download=pd.merge(summary_download, std_download,  how='outer', left_on=['SK_PI'], right_on = ['SK_PI'])

summary_download.fillna(0, inplace=True)
summary_download['Download speeds less than 50  Mbps  (%)']=round(summary_download['download_below50']/summary_download['download_count']*100)
summary_download=summary_download.sort_values(by=['SK_PI'], ascending=[True])
summary_download["device number"]=summary_download["SK_PI"]
summary_download=summary_download.reset_index().set_index("device number")
summary_download[["download_mean","download_std","Download speeds less than 50  Mbps  (%)"]]

,download_mean,download_std,Download speeds less than 50 Mbps (%)
device number,,,
5,5.95,0.87,100.0
7,11.46,2.37,100.0
8,101.27,15.69,0.0
9,38.84,9.02,99.0
10,3.96,1.34,100.0
11,29.21,7.97,99.0
12,14.97,3.31,100.0
14,108.91,56.63,32.0
15,17.28,3.65,100.0


In [16]:
device_number=14
subset=download_df[download_df["SK_PI"]==device_number]
hist_data = [subset['DOWNLOAD']]
group_labels = ['device '+str(device_number)+ ' download speed']
import plotly.figure_factory as ff
fig = ff.create_distplot(hist_data, group_labels,bin_size=10)
iplot(fig)

![](images/grafana_download14.png)

![](images/grafana_download14_1.png)

In [17]:
subset=download_df[download_df["SK_PI"]==device_number]
simple_boxplot(dataframe=subset,plot_value='DOWNLOAD',sort_value='hour',
               title="Download speed by hour for the device "+str(device_number)+" over the "+time_interval+" starting from "+starting_point,
               ytitle="Mbps",downloadline=True)

In [18]:
download_df["weekday"]=download_df["time"].dt.weekday_name
download_df["weekday"] = pd.Categorical(download_df["weekday"], ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])

In [19]:
subset=download_df[download_df["SK_PI"]==device_number]
simple_boxplot(dataframe=subset,plot_value='DOWNLOAD',sort_value='weekday',
               title="Download speed by day of the week for the device "+str(device_number)+" over the "+time_interval+" starting from "+starting_point,
               ytitle="Mbps",downloadline=True, weekdays=True, jitter=True)

![](images/plotly_testservers_14.png)

![](images/grafana_download14_3.png)

In [20]:
#devices_download_below50=summary_download[summary_download['download_below50_percent']>50]['SK_PI'].unique()
#hist_data=[]
#group_labels=[]
#colors_d=[]
#for device_number in devices_download_below50:
#    hist_data.append(download_df[download_df["SK_PI"]==device_number]["DOWNLOAD"])
#    group_labels.append(device_number)
#    colors_d.append(colors[device_number])

#fig = ff.create_distplot(hist_data, group_labels, colors=colors_d,
#                         bin_size=1, show_rug=False)
#
#fig['layout'].update(title='Download speed: distribution for all devices')
#iplot(fig)

## Upload speed

In [21]:
if not all_data:
    query_upload = "SELECT * FROM SPEEDTEST_UPLOAD WHERE PROVIDER!='iperf' AND time >= '"+starting_point+"'-"+time_interval+" AND UPLOAD>0;"
else:
    query_upload = "SELECT * FROM SPEEDTEST_UPLOAD WHERE PROVIDER!='iperf' AND UPLOAD>0;"
upload_df = get_dataframe_from_influxdb(client_df=client_df,query_influx=query_upload,table_name='SPEEDTEST_UPLOAD')
#upload_df.head()
upload_df=upload_df[upload_df["SK_PI"]!=3]
upload_df['time']= upload_df['time'].dt.tz_localize('UTC').dt.tz_convert('America/Winnipeg')

In [22]:
device_numbers_u=upload_df['SK_PI'].unique()
device_numbers_u=list(map(int, device_numbers_u))
device_numbers_u= sorted(device_numbers_u)
#device_numbers_u

In [23]:
upload_df["hour"]=pd.to_numeric(upload_df["time"].dt.hour)
by_hour_by_device_u = mean_max_median_min_by2(input_dataframe=upload_df,value1="UPLOAD", value2="UPLOAD",
                                              value3="UPLOAD",value4="UPLOAD",group_by_value="hour", rename_columns=True)

In [24]:
by_hour_by_device_u = mean_max_median_min_by2(input_dataframe=upload_df,value1="UPLOAD", value2="UPLOAD",
                                              value3="UPLOAD",value4="UPLOAD",group_by_value="hour", rename_columns=True)
traces=[]
for device in device_numbers_d:
    #subset=by_hour_by_device_u[by_hour_by_device_u["SK_PI"]==device]
    subset=upload_df[upload_df["SK_PI"]==device]
    trace = go.Scatter(
        x = subset['hour'],
        #y=subset['UPLOAD'],
        y=(subset['UPLOAD']-subset['UPLOAD'].mean())/subset['UPLOAD'].std(),
        #y=subset['mean'],
        #y=(subset['mean']-subset['mean'].mean())/subset['mean'].std(),
        mode = 'markers',
        marker = dict(color=colors[device]),
        name = device
    )
    traces.append(trace)
layout = go.Layout(
        title="Average upload speed by hour over the last "+time_interval+ " starting from "+ starting_point,
        xaxis=dict(title="hour"),
        yaxis=dict(title="  ")
        )
#download_line=go.Scatter(x=by_hour_by_device_d['hour'],y=[50] * len(by_hour_by_device_d['hour']), mode='lines',marker=dict(color='red'), name='50Mbps')
#traces.append(download_line)
data = traces
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [25]:
upload_summary=mean_max_median_min_by1(upload_df,'UPLOAD')

In [26]:
simple_boxplot(dataframe=upload_df,plot_value='UPLOAD',sort_value='SK_PI',
               title=t, 
               ytitle="Mbps",uploadline=True)

In [27]:
upload_df["below10"]=0
upload_df.loc[(upload_df["UPLOAD"]<10),"below10"]=1
#upload_df.head()

In [28]:
summary_upload=upload_df.groupby("SK_PI")['below10'].count().reset_index()
summary_upload.rename(columns={'below10':'upload_count'}, inplace=True)

mean_upload=round(upload_df.groupby("SK_PI")['UPLOAD'].mean(),2).reset_index()
mean_upload.rename(columns={'UPLOAD':'upload_mean'}, inplace=True)

std_upload=round(upload_df.groupby("SK_PI")['UPLOAD'].std(),2).reset_index()
std_upload.rename(columns={'UPLOAD':'upload_std'}, inplace=True)

subset_below10=upload_df[upload_df['below10']==1]
summary_below10=subset_below10.groupby("SK_PI")['below10'].count().reset_index()
summary_below10.rename(columns={'below10':'upload_below10'}, inplace=True)

summary_upload=pd.merge(summary_upload, summary_below10,  how='outer', left_on=['SK_PI'], right_on = ['SK_PI'])
summary_upload=pd.merge(summary_upload, mean_upload,  how='outer', left_on=['SK_PI'], right_on = ['SK_PI'])
summary_upload=pd.merge(summary_upload, std_upload,  how='outer', left_on=['SK_PI'], right_on = ['SK_PI'])

summary_upload.fillna(0, inplace=True)
summary_upload["Upload speeds less than 10  Mbps  (%)"]=round(summary_upload['upload_below10']/summary_upload['upload_count']*100)
summary_upload=summary_upload.sort_values(by=['SK_PI'], ascending=[ True])

summary_upload["device number"]=summary_upload["SK_PI"]
summary_upload=summary_upload.reset_index().set_index("device number")
summary_upload[["upload_mean","upload_std","Upload speeds less than 10  Mbps  (%)"]]

,upload_mean,upload_std,Upload speeds less than 10 Mbps (%)
device number,,,
5,0.51,0.06,100.0
7,8.18,2.63,76.0
8,96.37,19.14,0.0
9,48.74,10.27,0.0
10,4.03,0.96,100.0
11,54.23,7.08,0.0
12,7.48,1.44,99.0
14,126.04,24.97,1.0
15,43.80,2.43,0.0


In [29]:
device_number=7
subset=upload_df[upload_df["SK_PI"]==device_number]
hist_data = [subset['UPLOAD']]
group_labels = ['device '+str(device_number)+ ' upload speed']
import plotly.figure_factory as ff
fig = ff.create_distplot(hist_data, group_labels,bin_size=0.25)
iplot(fig)


![](images/grafana_upload7.png)

In [30]:
upload_df["hour"]=pd.to_numeric(upload_df["time"].dt.hour)
subset=upload_df[upload_df["SK_PI"]==device_number]
simple_boxplot(dataframe=subset,plot_value='UPLOAD',sort_value='hour',
               title="Upload speed by hour for the device "+str(device_number)+" over the "+time_interval+" starting from "+starting_point,
               ytitle="Mbps",uploadline=True)

In [31]:
upload_df["time_group"]=""
upload_df.loc[(upload_df["hour"]>23)|(upload_df["hour"]<=7),"time_group"]="night 23:00-07:00"
upload_df.loc[(upload_df["hour"]>7)&(upload_df["hour"]<=17),"time_group"]="day 7:00-17:00"
upload_df.loc[(upload_df["hour"]>17)&(upload_df["hour"]<=23),"time_group"]="evening 17:00-23:00"

In [32]:
subset=upload_df[upload_df["SK_PI"]==device_number]
simple_boxplot(dataframe=subset,plot_value='UPLOAD',sort_value='time_group',
               title="Upload speed by timegroup for device "+str(device_number)+" over the "+time_interval+" starting from "+starting_point,
               ytitle="Mbps",uploadline=True, jitter=True)

In [33]:
upload_df["weekday"]=upload_df["time"].dt.weekday_name
upload_df["weekday"] = pd.Categorical(upload_df["weekday"], ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])

In [34]:
subset=upload_df[upload_df["SK_PI"]==device_number]
simple_boxplot(dataframe=subset,plot_value='UPLOAD',sort_value='weekday',
               title="Upload speed by day of the week for the device "+str(device_number)+" over the "+time_interval+" starting from "+starting_point,
               ytitle="Mbps",uploadline=True, weekdays=True, jitter=True)

In [35]:
upload_df["day_group"]="Weekday"
upload_df.loc[(upload_df["weekday"]=="Sunday")|(upload_df["weekday"]=="Saturday"),"day_group"]="Weekend"
subset=upload_df[upload_df["SK_PI"]==device_number]
simple_boxplot(dataframe=subset,plot_value='UPLOAD',sort_value='day_group',
               title="Upload speed by day group for device "+str(device_number)+" over the "+time_interval+" starting from "+starting_point,
               ytitle="Mbps",uploadline=True, jitter=True)

In [36]:
#devices_upload_below10=summary_upload[summary_upload['upload_below10_percent']>50]['SK_PI'].unique()
#hist_data=[]
#group_labels=[]
#colors_d=[]
#for device_number in devices_upload_below10:
#    hist_data.append(upload_df[upload_df["SK_PI"]==device_number]["UPLOAD"])
#    group_labels.append(device_number)
#    colors_d.append(colors[device_number])
#
#fig = ff.create_distplot(hist_data, group_labels, colors=colors_d,
#                         bin_size=0.25, show_rug=False)
#
# Add title
#fig['layout'].update(title='Upload speed: distribution for all devices')
#iplot(fig)

In [37]:
summary_upload_download=pd.merge(summary_upload, summary_download,  how='outer', left_on=['SK_PI'], right_on = ['SK_PI'])
summary_upload_download=summary_upload_download.sort_values(by=['Download speeds less than 50  Mbps  (%)','Upload speeds less than 10  Mbps  (%)','SK_PI'], ascending=[False, False, True])
summary_upload_download[['Upload speeds less than 10  Mbps  (%)','Download speeds less than 50  Mbps  (%)']]

,Upload speeds less than 10 Mbps (%),Download speeds less than 50 Mbps (%)
0,100.0,100.0
4,100.0,100.0
9,100.0,100.0
6,99.0,100.0
11,98.0,100.0
1,76.0,100.0
8,0.0,100.0
10,0.0,100.0
3,0.0,99.0
5,0.0,99.0


In [55]:
d = {'SK_PI': [7,12,14,17,11,16,10,18], 'lat': [51.7834662, 50.366367,49.81892,56.5020674,49.13794,
                                                  50.533,50.9080366,56.0867787], 'long': [-96.6966330,-96.613671,
                                                -98.52097,-94.2086756,-97.24325,-100.3155,-98.5971688,-96.0936590],
                                                'name':['Bloodvein','Brokenhead','Dakota Plains','Fox Lake','Ginew',
                                                        'Keeseekoowenin','Lake Manitoba','York Landing']}
coordinates_df = pd.DataFrame(data=d)
summary_ud_loc=pd.merge(summary_upload_download, coordinates_df,  how='right', left_on=['SK_PI'], right_on = ['SK_PI'])
summary_ud_loc

,index_x,SK_PI,upload_count,upload_below10,upload_mean,upload_std,Upload speeds less than 10 Mbps (%),index_y,download_count,download_below50,download_mean,download_std,Download speeds less than 50 Mbps (%),lat,long,name
0,4,10,154,154.0,4.03,0.96,100.0,4,154,154.0,3.96,1.34,100.0,50.908037,-98.597169,Lake Manitoba
1,9,16,161,161.0,3.74,0.31,100.0,9,161,161.0,5.52,1.39,100.0,50.533000,-100.315500,Keeseekoowenin
2,6,12,172,170.0,7.48,1.44,99.0,6,172,172.0,14.97,3.31,100.0,50.366367,-96.613671,Brokenhead
3,11,18,168,164.0,9.55,0.22,98.0,11,168,168.0,9.25,0.60,100.0,56.086779,-96.093659,York Landing
4,1,7,172,131.0,8.18,2.63,76.0,1,172,172.0,11.46,2.37,100.0,51.783466,-96.696633,Bloodvein
5,10,17,170,0.0,46.30,3.64,0.0,10,170,170.0,37.51,5.38,100.0,56.502067,-94.208676,Fox Lake
6,5,11,172,0.0,54.23,7.08,0.0,5,172,171.0,29.21,7.97,99.0,49.137940,-97.243250,Ginew
7,7,14,171,2.0,126.04,24.97,1.0,7,171,54.0,108.91,56.63,32.0,49.818920,-98.520970,Dakota Plains


In [54]:
summary_ud_loc['text1'] = summary_ud_loc['SK_PI'].astype(str)+'.'+summary_ud_loc['name'] +': Average download speed '+\
                          summary_ud_loc['download_mean'].astype(str) + 'Mbps , standart deviaton: '+\
                          summary_ud_loc['download_std'].astype(str)
data = [ dict(
        type = 'scattergeo',
        locationmode = 'north america',
        lon = summary_ud_loc['long'],
        lat = summary_ud_loc['lat'],
        text = summary_ud_loc['text1'],
        mode = 'markers',
        marker = dict(
            size = 8,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = False,
            symbol = 'circle',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
            colorscale='Blues',
            #colorscale = scl,
            cmin = 0,
            color = summary_ud_loc['download_mean'],
            cmax = summary_ud_loc['download_mean'].max(),
            colorbar=dict(
               # title="Percentage of download speed data below 50Mbs"
            )
        ))]

layout = dict(
        title = 'Devices colored by average download speed',
        colorbar = True,
        geo = dict(
            scope = 'north america',
            showland = True,
            landcolor = "rgb(212, 212, 212)",
            countrycolor = "rgb(255, 255, 255)",
            showlakes = True,
            lakecolor = "rgb(255, 255, 255)",
            showsubunits = True,
            showcountries = True,
            resolution = 50,
            projection = dict(
                type = 'kavrayskiy7',
            ),
             lonaxis = dict(
                gridwidth = 2,
                range= [ -110, -80 ],
                dtick = 10
            ),
            lataxis = dict (
                range= [ 47.0, 60.0 ],
                dtick = 10
            )
        ),
    )

fig = dict( data=data, layout=layout )
iplot( fig, validate=False)

In [40]:
summary_ud_loc['text'] = summary_ud_loc['SK_PI'].astype(str)+'.'+summary_ud_loc['name'] +': Download speed: '+\
                         summary_ud_loc['Download speeds less than 50  Mbps  (%)'].astype(str)+\
                        '% of the data is below 50Mbps'
data = [ dict(
        type = 'scattergeo',
        locationmode = 'north america',
        lon = summary_ud_loc['long'],
        lat = summary_ud_loc['lat'],
        text = summary_ud_loc['text'],
        mode = 'markers',
        marker = dict(
            size = 8,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = False,
            symbol = 'circle',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
            colorscale='Greens',
            #colorscale = scl,
            cmin = 0,
            color = summary_ud_loc['Download speeds less than 50  Mbps  (%)'],
            cmax = summary_ud_loc['Download speeds less than 50  Mbps  (%)'].max(),
            colorbar=dict(
               # title="Percentage of download speed data below 50Mbs"
            )
        ))]

layout = dict(
        title = 'Devices colored by percentage of download speed data less than 50  Mbps',
        colorbar = True,
        geo = dict(
            scope = 'north america',
            showland = True,
            landcolor = "rgb(212, 212, 212)",
            countrycolor = "rgb(255, 255, 255)",
            showlakes = True,
            lakecolor = "rgb(255, 255, 255)",
            showsubunits = True,
            showcountries = True,
            resolution = 50,
            projection = dict(
                type = 'kavrayskiy7',
            ),
              lonaxis = dict(
                gridwidth = 2,
                range= [ -110, -80 ],
                dtick = 10
            ),
            lataxis = dict (
                range= [ 47.0, 60.0 ],
                dtick = 10
            )
        ),
    )

fig = dict( data=data, layout=layout )
iplot( fig, validate=False)

Areas to Enhance Broadband Access# map on CRTC website isavailable here: https://crtc.gc.ca/eng/internet/band.htm

![](images/manitoba_crtc.png)

### Ping latency

In [41]:
device_numbers=get_tag_values_influxdb(client_influx=client,table_name='PING', tag_name='SK_PI')
device_numbers=list(map(int, device_numbers))
device_numbers= sorted(device_numbers)
ping_mean_query="SELECT MAX(PING),MEAN(PING), MEDIAN(PING) FROM PING WHERE PING!=0 AND time >= '"+starting_point+"'-"+\
                time_interval+" GROUP BY time(1h), SK_PI;"

In [42]:
ping_df=get_3_stats_influxdb(client_influx=client,
                                           query_influx=ping_mean_query,
                                           stat_name1='max',
                                           stat_name2='mean',
                                           stat_name3='median',
                                           device_numbers=device_numbers)
ping_df=ping_df[ping_df["SK_PI"]!=3]
ping_df['time']= ping_df['time'].dt.tz_localize('UTC').dt.tz_convert('America/Winnipeg')

In [43]:
device_numbers_p=ping_df['SK_PI'].unique()
device_numbers_p=list(map(int, device_numbers_p))
device_numbers_p= sorted(device_numbers_p)
#device_numbers

In [44]:
ping_df["hour"]=pd.to_numeric(ping_df["time"].dt.hour)

In [45]:
simple_boxplot(dataframe=ping_df,plot_value='mean',sort_value='hour',
               title="Ping latency by hour for all the devices over the "+time_interval+" starting from "+starting_point,
               ytitle="Miliseconds")

In [46]:
ping_df["weekday"]=ping_df["time"].dt.weekday_name
ping_df["weekday"] = pd.Categorical(ping_df["weekday"], ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])

In [47]:
simple_boxplot(dataframe=ping_df,plot_value='mean',sort_value='weekday',
               title="Ping latency by hour for all the devices over the "+time_interval+" starting from "+starting_point,
               ytitle="Miliseconds")

In [48]:
by_hour_by_device_p1=mean_max_median_by2(input_dataframe=ping_df,value1="mean", value2="max",
                                          value3="median",group_by_value="hour")
device_number=7
subset=by_hour_by_device_p1[by_hour_by_device_p1["SK_PI"]==device_number]
combined_bar_plot_3traces(xvalues=subset["hour"],
                         yvalues1=subset["max"],
                         yvalues2=subset["mean"],
                         yvalues3=subset["median"],
                         name1="Max",
                         name2="Mean",
                         name3="Median",
                         title="Ping latency(collectd) by hour for the device "+str(device_number)+" over the "+time_interval+" starting from "+starting_point,
                         xtitle="hour",
                         stack=False)#,updatemenus=updatemenus1, annotations=annotations1)

In [49]:
query_ping = "SELECT * FROM SPEEDTEST_PING WHERE PROVIDER!='iperf' AND time >='"+starting_point+"'-"+time_interval+" AND PING!=0;"
ping_speedtest_dataframe = get_dataframe_from_influxdb(client_df=client_df,query_influx=query_ping,table_name='SPEEDTEST_PING')

In [50]:
ping_speedtest_dataframe=ping_speedtest_dataframe[ping_speedtest_dataframe["SK_PI"]!=3]
ping_speedtest_dataframe['time']= ping_speedtest_dataframe['time'].dt.tz_localize('UTC').dt.tz_convert('America/Winnipeg')

In [51]:
ping_speedtest_dataframe["hour"]=pd.to_numeric(ping_speedtest_dataframe["time"].dt.hour)

In [52]:
by_hour_by_device_p2=mean_max_median_by2(input_dataframe=ping_speedtest_dataframe,value1="PING", value2="PING",
                                          value3="PING",group_by_value="hour", rename_columns=True)

In [53]:
device_number=14
subset=by_hour_by_device_p2[by_hour_by_device_p2["SK_PI"]==device_number]
combined_bar_plot_3traces(xvalues=subset["hour"],
                         yvalues1=subset["max"],
                         yvalues2=subset["mean"],
                         yvalues3=subset["median"],
                         name1="Max",
                         name2="Mean",
                         name3="Median",
                         title="Ping latency(speedtest) by hour for the device "+str(device_number)+" over the "+time_interval+" starting from "+starting_point,
                         xtitle="hour",
                         stack=False)